In [3]:
import pyodbc
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import os
import httplib2
from apiclient import discovery
from google.oauth2 import service_account
# https://github.com/mkleehammer/pyodbc/wiki
from datetime import datetime, timezone, timedelta
import pandas as pd
from pytz import timezone

# tz = timezone('America/Los_Angeles')
# data3 = datetime.today().astimezone().strftime("%Y-%m-%d %H:%M:%S %p %Z") # needs pytz
# data4 = datetime.today().strftime("%B %m, %Y - %I:%M %p PDT")

In [4]:
def sendEmail():
    email = os.environ.get("USER_EMAIL") # the email where you sent the email
    password = os.environ.get("USER_PW")
    send_to_email = os.environ.get("USER_EMAIL") # for whom
    message = """\
<html>
  <head></head>
  <body>
    <p>Hi!<br> <br>
       How are you?<br>
       Here is the <a href="https://docs.google.com/spreadsheets/d/1iVCGoVT6JsuRnNY7sR38e_yZxVOQZLZibmbeNGeyJgU/edit#gid=1443099774">link</a> you wanted.
       <br> <br>
       Jacob
    </p>
  </body>
</html>
"""
    msg = MIMEMultipart()
    msg["From"] = email
    msg["To"] = send_to_email
    msg["Subject"] = "Gmail prac from Python"
    msg.attach(MIMEText(message, 'html'))

    server = smtplib.SMTP("smtp.gmail.com", 587)
    server.starttls()
    server.login(email, password)
    text = msg.as_string()
    server.sendmail(email, send_to_email, text)
    server.quit()

In [5]:
## spreadsheet stuff
try: 
    scopes = ["https://www.googleapis.com/auth/drive", "https://www.googleapis.com/auth/drive.file",
              "https://www.googleapis.com/auth/spreadsheets"]
    secret_file = os.path.join(os.getcwd(), "key/jacobs-googledrive.json")

    spreadsheet_id = '1iVCGoVT6JsuRnNY7sR38e_yZxVOQZLZibmbeNGeyJgU'
    # range_name = 'current_time'
    range_name = 'current_time!A1'

    credentials = service_account.Credentials.from_service_account_file(secret_file, scopes = scopes)
    service = discovery.build('sheets', 'v4', credentials=credentials)

    values = [[datetime.today().strftime("%B %d, %Y - %I:%M %p CST")]]

    data1 = {
        "values": values
    }
    #service.spreadsheets().values() is basically always the same
    #you can do .update() or .append() after that
        # with spreadsheetId = spread
    service.spreadsheets().values().append(spreadsheetId=spreadsheet_id, body=data1, range=range_name,
                                           valueInputOption='USER_ENTERED').execute()

    sendEmail()

except OSError as e:
    print(e)